In [1]:
import os
os.chdir('..')

os.environ["DJANGO_SETTINGS_MODULE"] = "course_management.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ["IS_LOCAL"] = "1"

import django
django.setup()

IS_LOCAL=True
['C:\\Users\\alexe\\.virtualenvs\\course-management-platform-wiAsnpQu\\Lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\alexe\\AppData\\Roaming\\jupyter\\runtime\\kernel-8b131781-a55c-45fb-b820-d523bc533a5e.json']
Is test: False


In [2]:
from courses.models import *

In [3]:
from courses.scoring import update_faq_score, update_learning_in_public_score, update_leaderboard

In [4]:
from django.db.models import Prefetch

In [5]:
course = Course.objects.get(id=5)
course

<Course: Machine Learning Zoomcamp 2024>

In [6]:
homework = Homework.objects.get(course=course, slug='article')
homework

<Homework: Machine Learning Zoomcamp 2024 - Article>

In [7]:
answers_prefetch = Prefetch("answer_set", queryset=Answer.objects.all())

In [8]:
submissions = Submission.objects.prefetch_related(answers_prefetch).filter(homework=homework)

In [25]:
for s in submissions:
    print(s.id) #, s.student)
    
    answers = list(s.answer_set.values("id", "question_id", "answer_text"))
    print(answers)

    print(s.homework_link)
    break

13371
[{'id': 74388, 'question_id': 201, 'answer_text': 'https://medium.com/@edosomapatrick41/beware-of-the-accuracy-trap-4785d07cdd8a'}]



## Custom logic

In [19]:
for s in submissions:
    answers_raw = list(s.answer_set.all())
    answers = {d.question_id: d for d in answers_raw}

    answer_url = answers[201]
    url = answer_url.answer_text
    url2 = s.homework_link

    url = url2 or url

    print(s.id, url)

13371 https://medium.com/@edosomapatrick41/beware-of-the-accuracy-trap-4785d07cdd8a
13376 https://github.com/DataTalksClub/machine-learning-zoomcamp/blob/master/cohorts/2024/01-intro/homework.md
14638 https://medium.com/@ket14pril14/top-machine-learning-optimization-algorithms-9063447198a1
15381 https://medium.com/@markus_datadude/lightgbm-vs-catboost-vs-xgboost-the-ultimate-guide-to-faster-and-smarter-machine-learning-1a7f399d4c4f
15370 https://github.com/3d150n-marc3l0/ml-zoomcamp/blob/main/article/articule.md
16291 https://bankymondial.github.io/Beyond-Model-Selection/
13369 https://markogolovko.com/blog/you-will-like-my-offer/
15663 https://github.com/yann-pham-van/DataEngineeringZoomcamp/blob/main/kestra_data_warehouse_dlt_chatgpt.md
14982 https://github.com/MarDom15/Publication_paper/blob/main/Dimensionality%20Reduction%20in%20Machine%20Learning_A%20Comprehensive%20Analysis%20of%20Techniques%20and%20Applications_/paperdata_zoomcamp.pdf
15442 https://github.com/adikusum4/Machine-L

In [21]:
invalid = {13376, 15663, }

In [22]:
for s in submissions:
    print(s.id)
    
    answers_raw = list(s.answer_set.all())

    points = 20
    if s.id in invalid:
        points = 0

    s.questions_score = points

    lip_score = update_learning_in_public_score(s)
    faq_score = update_faq_score(s)
    
    s.total_score = points + lip_score + faq_score
    s.save()

13371
13376
14638
15381
15370
16291
13369
15663
14982
15442
15390
15458
15711
15481
16011
15552
15854


In [23]:
homework.state = HomeworkState.SCORED.value
homework.save()

In [24]:
update_leaderboard(course)

INFO Updating leaderboard for course 5
INFO Updated leaderboard in 1.66 seconds
